In [1]:
textgrid_dir = "/Users/joregan/Playing/textgrids_shared"
timecode_dir = "/Users/joregan/timecode_cut"

In [2]:
from pathlib import Path

textgrid_path = Path(textgrid_dir)
timecode_path = Path(timecode_dir)

In [3]:
def get_timecode_offsets(filename):
    if type(filename) == Path:
        filename = str(filename)

    with open(filename) as inf:
        lines = [l.strip() for l in inf.readlines()]
        assert lines[0] == ",Frame,Time (Seconds),TimeCode", f"CSV file ({filename}) seems to be incorrect"
        p_start = lines[1].split(",")
        start = float(p_start[2])
        p_end = lines[-1].split(",")
        end = float(p_end[2])
        return start, end

In [18]:
from praatio import textgrid
from praatio.utilities.constants import Interval

def modify_textgrid(filename, start, end, output=None):
    if type(filename) == Path:
        filename = str(filename)
    if not output:
        output = filename
    tg = textgrid.openTextgrid(filename, True)
    new_tg = textgrid.Textgrid()
    for tiername in tg.tierNames:
        tier = tg.getTier(tiername)
        new_entries = []
        for entry in tier.entries:
            if entry[0] < start and entry[1] < start:
                continue
            elif entry[0] > end and entry[1] > end:
                continue
            elif entry[0] <= start and entry[1] > start:
                if entry[2] != "":
                    print("Warning: truncating entry", entry)
                if new_entries == []:
                    new_entries.append(Interval(0.0, entry[1] - start, entry[2]))
                else:
                    print("Shouldn't have existing entries!!", entry)
                    new_entries.append(Interval(entry[0] - start, entry[1] - start, entry[2]))
            elif entry[0] >= start and entry[1] <= end:
                new_entries.append(Interval(entry[0] - start, entry[1] - start, entry[2]))
            elif entry[0] <= end and entry[1] > end:
                if entry[2] != "":
                    print("Warning: truncating entry", entry)
                new_entries.append(Interval(entry[0] - start, entry[1] - start, entry[2]))
            else:
                print("There should be no default case", entry)
        tier_start = new_entries[0][0]
        tier_end = new_entries[-1][1]
        new_tier = textgrid.IntervalTier(tiername, new_entries, tier_start, tier_end)
        new_tg.addTier(new_tier)
    tg.save(output, format="long_textgrid", includeBlankSpaces=True)

In [11]:
tmp = textgrid.openTextgrid("/Users/joregan/Playing/textgrids_shared/hsi_6_0718_211_001_main.TextGrid", True)

In [14]:
get_timecode_offsets("/Users/joregan/timecode_cut/hsi_6_0718_211_001.csv")

(13.283, 398.042)

In [19]:
modify_textgrid("/Users/joregan/Playing/textgrids_shared/hsi_6_0718_211_001_main.TextGrid", 13.283, 398.042, "/tmp/test.TextGrid")